In [172]:
import pandas as pd
import numpy as np
import datetime as dt
from bs4 import BeautifulSoup
import requests

In [173]:
#cargamos los 3 datasets que vamos a utlizzar para crear las funciones
dfjuegos=pd.read_csv('Juegos_steam.csv')
dfreviews=pd.read_csv('revies_por_usuario.csv')
dfitems=pd.read_csv('items_por_usuario.csv')

In [174]:
#def PlayTimeGenre( genero : str ): #Debe devolver año con mas horas jugadas para dicho género.
#Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

In [175]:
#por ahora trabajmos con los años que si tenemos datos, despues se vera el tratamiento de los años restantes
dfjuegos=dfjuegos.dropna(subset='Año_lanzamiento')

In [176]:
#hacemos una lista con los generos para poder acceder a ellos
generos=[]
gen=dfjuegos['Género'].drop_duplicates().str.strip("']['").str.split(", ",expand=True).copy()
for a in range(len(gen.columns)):
    for b in gen[a].unique():
        if b !=None:
            if b.strip("' ") not in generos:
                if b.strip("' ")!='':
                    generos.append(b.strip("' "))
              

In [177]:
# ordenamos por alfabeto
generos=sorted(generos)
generos

['Accounting',
 'Action',
 'Adventure',
 'Animation & Modeling',
 'Audio Production',
 'Casual',
 'Design & Illustration',
 'Early Access',
 'Education',
 'Free to Play',
 'Game Development',
 'Indie',
 'Massively Multiplayer',
 'N/D',
 'Photo Editing',
 'RPG',
 'Racing',
 'Simulation',
 'Software Training',
 'Sports',
 'Strategy',
 'Utilities',
 'Video Production',
 'Web Publishing']

In [178]:
# primero sumamos las horas de cada juego en total
itemsf1=dfitems[['Id_juego','Horas_jugadas']]
dff1=itemsf1.groupby('Id_juego').sum()
dff1.head()

,Horas_jugadas
Id_juego,
10,17107858
20,960524
30,756375
40,154424
50,726545


In [179]:
#añadimos las columnas de todos los generos, un genero por columna, se llena con 0 por default
for a in generos:
    dff1[a]=np.zeros(len(dff1),dtype=int)
#se agrega la columna Género al dataframe

dff1.head()

,Horas_jugadas,Accounting,Action,Adventure,Animation & Modeling,Audio Production,Casual,Design & Illustration,Early Access,Education,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
Id_juego,,,,,,,,,,,,,,,,,,,,,
10,17107858,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20,960524,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30,756375,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40,154424,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
50,726545,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [180]:
#agregamos la columna generos, y rellenamos con 'N/D' por default, en caso que haya ids que no esten en nuestro dataset de juegos
Generos=['N/D' for a in range(len(dff1))]
dff1['Género']=Generos
dff1.Género

Id_juego
10        N/D
20        N/D
30        N/D
40        N/D
50        N/D
         ... 
528580    N/D
528660    N/D
529670    N/D
529820    N/D
530720    N/D
Name: Género, Length: 10978, dtype: object

In [181]:
#rellenamos la columna Género con los generos del dataframe dfjuegos
for b,a in enumerate(dff1.index):
    if dfjuegos.Id_juego[dfjuegos.Id_juego==a].array.size>0:
        dff1.loc[a,'Género']=dfjuegos.Género[dfjuegos.Id_juego==a].iloc[0]
dff1.head()

,Horas_jugadas,Accounting,Action,Adventure,Animation & Modeling,Audio Production,Casual,Design & Illustration,Early Access,Education,...,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing,Género
Id_juego,,,,,,,,,,,,,,,,,,,,,
10,17107858,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,['Action']
20,960524,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,['Action']
30,756375,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,['Action']
40,154424,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,['Action']
50,726545,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,['Action']


In [182]:
#ahora rellenamos las columnas de cada genero, donde el genero si exista en la colomna Género
for a in generos:
    dff1.loc[dff1.Género.str.contains(a),a]=1
dff1.head()

,Horas_jugadas,Accounting,Action,Adventure,Animation & Modeling,Audio Production,Casual,Design & Illustration,Early Access,Education,...,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing,Género
Id_juego,,,,,,,,,,,,,,,,,,,,,
10,17107858,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,['Action']
20,960524,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,['Action']
30,756375,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,['Action']
40,154424,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,['Action']
50,726545,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,['Action']


In [183]:
#agregamos la columna Año, y rellenamos con 'N/D' por default, en caso que haya ids que no esten en nuestro dataset de juegos
años=['N/D' for a in range(len(dff1))]
dff1['Año']=años
dff1.Año

Id_juego
10        N/D
20        N/D
30        N/D
40        N/D
50        N/D
         ... 
528580    N/D
528660    N/D
529670    N/D
529820    N/D
530720    N/D
Name: Año, Length: 10978, dtype: object

In [184]:
# Agregamos la columna año al dataframe de la funcion
for b,a in enumerate(dff1.index):
    if dfjuegos.Id_juego[dfjuegos.Id_juego==a].array.size>0:
        dff1.loc[a,'Año']=dfjuegos.Año_lanzamiento[dfjuegos.Id_juego==a].iloc[0]
dff1.head()

,Horas_jugadas,Accounting,Action,Adventure,Animation & Modeling,Audio Production,Casual,Design & Illustration,Early Access,Education,...,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing,Género,Año
Id_juego,,,,,,,,,,,,,,,,,,,,,
10,17107858,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,['Action'],2000
20,960524,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,['Action'],1999
30,756375,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,['Action'],2003
40,154424,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,['Action'],2001
50,726545,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,['Action'],1999


In [185]:
dff1.columns

Index(['Horas_jugadas', 'Accounting', 'Action', 'Adventure',
       'Animation & Modeling', 'Audio Production', 'Casual',
       'Design & Illustration', 'Early Access', 'Education', 'Free to Play',
       'Game Development', 'Indie', 'Massively Multiplayer', 'N/D',
       'Photo Editing', 'RPG', 'Racing', 'Simulation', 'Software Training',
       'Sports', 'Strategy', 'Utilities', 'Video Production', 'Web Publishing',
       'Género', 'Año'],
      dtype='object')

In [186]:
#reordenamos las columnas
dff1=dff1[['Género', 'Año','Horas_jugadas', 'Accounting', 'Action', 'Adventure',
       'Animation & Modeling', 'Audio Production', 'Casual',
       'Design & Illustration', 'Early Access', 'Education', 'Free to Play',
       'Game Development', 'Indie', 'Massively Multiplayer', 'N/D',
       'Photo Editing', 'RPG', 'Racing', 'Simulation', 'Software Training',
       'Sports', 'Strategy', 'Utilities', 'Video Production', 'Web Publishing',]]
# confirmamos orden
dff1.columns

Index(['Género', 'Año', 'Horas_jugadas', 'Accounting', 'Action', 'Adventure',
       'Animation & Modeling', 'Audio Production', 'Casual',
       'Design & Illustration', 'Early Access', 'Education', 'Free to Play',
       'Game Development', 'Indie', 'Massively Multiplayer', 'N/D',
       'Photo Editing', 'RPG', 'Racing', 'Simulation', 'Software Training',
       'Sports', 'Strategy', 'Utilities', 'Video Production',
       'Web Publishing'],
      dtype='object')

In [187]:
dff1.head()

,Género,Año,Horas_jugadas,Accounting,Action,Adventure,Animation & Modeling,Audio Production,Casual,Design & Illustration,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
Id_juego,,,,,,,,,,,,,,,,,,,,,
10,['Action'],2000,17107858,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20,['Action'],1999,960524,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30,['Action'],2003,756375,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40,['Action'],2001,154424,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
50,['Action'],1999,726545,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [188]:
# se hace un Dataframe vacio para guardar las columnas filtradas
dff1_final=pd.DataFrame()

In [189]:
#se agrupa por año y genero, sumando las horas por genero en cada año
for a in generos:
    if dff1[dff1[a]==1].groupby(by=['Año']).sum()['Horas_jugadas'].array.size==0:
        continue
    else:
        dftemp1=dff1[dff1[a]==1].groupby(by=['Año']).sum()['Horas_jugadas'].reset_index()
        gen=[a for b in range(len(dftemp1))]
        dftemp1['Género']=pd.Series(gen)
        dftemp1=dftemp1.sort_values(by='Horas_jugadas',ascending=False)
        dftemp1=dftemp1.iloc[[0]]
        dff1_final=pd.concat([dff1_final,dftemp1])
 
dff1_final

,Año,Horas_jugadas,Género
0,2016,0,Accounting
26,2012,1085700047,Action
26,2011,221804836,Adventure
4,2015,1345913,Animation & Modeling
2,2014,455463,Audio Production
26,2015,83950877,Casual
2,2012,1930339,Design & Illustration
1,2013,118902893,Early Access
2,2013,340567,Education
12,2013,189185785,Free to Play


In [190]:
# guardamos en un csv el datafrme listo para la funcion
dff1_final.to_csv('Función_PlayTimeGenre.csv',index=False)

In [191]:
def PlayTimeGenre(genero):
    dict={}
    genero=genero.capitalize()
    mensaje="Año de lanzamiento con más horas jugadas para Género "+genero
    dict[mensaje]=dff1_final.Año[dff1_final['Género']==genero].iloc[0]
    return dict

In [192]:
PlayTimeGenre('action')

{'Año de lanzamiento con más horas jugadas para Género Action': '2012'}

In [193]:

#def UserForGenre( genero : str ): #Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.
#Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [194]:
# primero sumamos las horas de cada juego en total
itemsf2=dfitems[['Id_usuario','Id_juego','Horas_jugadas']]
itemsf2.head()



,Id_usuario,Id_juego,Horas_jugadas
0,76561197970982479,10,6
1,76561197970982479,20,0
2,76561197970982479,30,7
3,76561197970982479,40,0
4,76561197970982479,50,0


In [195]:
#agregamos la columna generos, y rellenamos con 'N/D' por default, en caso que haya ids que no esten en nuestro dataset de juegos
Generos=['N/D' for a in range(len(itemsf2))]
itemsf2.loc[:,'Género']=Generos
itemsf2.Género

0          N/D
1          N/D
2          N/D
3          N/D
4          N/D
          ... 
5094087    N/D
5094088    N/D
5094089    N/D
5094090    N/D
5094091    N/D
Name: Género, Length: 5094092, dtype: object

In [196]:
#rellenamos la columna Género con los generos del dataframe dfjuegos
for a in itemsf2.Id_juego.unique():
    if dfjuegos.Id_juego[dfjuegos.Id_juego==a].array.size>0:
        itemsf2.loc[itemsf2.Id_juego==a,'Género']=dfjuegos.Género[dfjuegos.Id_juego==a].iloc[0]

In [197]:
#agregamos la columna Año, y rellenamos con 'N/D' por default, en caso que haya ids que no esten en nuestro dataset de juegos
años=['N/D' for a in range(len(itemsf2))]
itemsf2['Año']=años
itemsf2.Año

0          N/D
1          N/D
2          N/D
3          N/D
4          N/D
          ... 
5094087    N/D
5094088    N/D
5094089    N/D
5094090    N/D
5094091    N/D
Name: Año, Length: 5094092, dtype: object

In [198]:
# rellenamos la columna año en el dataframe de la funcion 2
for a in itemsf2.Id_juego.unique():
    if dfjuegos.Id_juego[dfjuegos.Id_juego==a].array.size>0:
        itemsf2.loc[itemsf2.Id_juego==a,'Año']=dfjuegos.Año_lanzamiento[dfjuegos.Id_juego==a].iloc[0]
itemsf2.head()

,Id_usuario,Id_juego,Horas_jugadas,Género,Año
0,76561197970982479,10,6,['Action'],2000
1,76561197970982479,20,0,['Action'],1999
2,76561197970982479,30,7,['Action'],2003
3,76561197970982479,40,0,['Action'],2001
4,76561197970982479,50,0,['Action'],1999


In [199]:
#añadimos las columnas de todos los generos, un genero por columna, se llena con 0 por default
for a in generos:
   itemsf2[a]=np.zeros(len(itemsf2),dtype=int)
#se agrega la columna Género al dataframe

itemsf2.head()

,Id_usuario,Id_juego,Horas_jugadas,Género,Año,Accounting,Action,Adventure,Animation & Modeling,Audio Production,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
0,76561197970982479,10,6,['Action'],2000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,76561197970982479,20,0,['Action'],1999,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,76561197970982479,30,7,['Action'],2003,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,76561197970982479,40,0,['Action'],2001,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,76561197970982479,50,0,['Action'],1999,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [200]:
#ahora rellenamos las columnas de cada genero, donde el genero si exista en la colomna Género
for a in generos:
    itemsf2.loc[itemsf2.Género.str.contains(a),a]=1
itemsf2.head()

,Id_usuario,Id_juego,Horas_jugadas,Género,Año,Accounting,Action,Adventure,Animation & Modeling,Audio Production,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
0,76561197970982479,10,6,['Action'],2000,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,76561197970982479,20,0,['Action'],1999,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,76561197970982479,30,7,['Action'],2003,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,76561197970982479,40,0,['Action'],2001,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,76561197970982479,50,0,['Action'],1999,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [201]:
# se crea un dataframe vacio para guardar las columnas agrupadas
dff2=pd.DataFrame()

In [202]:
#se agrupa por año y genero, sumando las horas por genero en cada año
for a in generos:
    if itemsf2[itemsf2[a]==1].groupby(by=['Id_usuario','Año']).sum()['Horas_jugadas'].array.size==0:
        continue
    else:
        dftemp1=pd.DataFrame(itemsf2[itemsf2[a]==1].groupby(by=['Id_usuario','Año']).sum()['Horas_jugadas']).reset_index()
        dftemp2=pd.DataFrame(itemsf2[itemsf2[a]==1].groupby(by=['Id_usuario']).sum()['Horas_jugadas']).reset_index()
        usuario=dftemp2.sort_values(by='Horas_jugadas',ascending=False).iloc[0,0]
        genero=[a for b in range(len(dftemp1[dftemp1.Id_usuario==usuario]))]
        dftemp3=dftemp1[dftemp1.Id_usuario==usuario].copy()
        dftemp3.loc[:,'Género']=genero
        

        dff2=pd.concat([dff2,dftemp3])
dff2

,Id_usuario,Año,Horas_jugadas,Género
0,76561198042410388,2016,0,Accounting
483994,Sp3ctre,1993,0,Action
483995,Sp3ctre,1995,217,Action
483996,Sp3ctre,1996,0,Action
483997,Sp3ctre,1998,0,Action
...,...,...,...,...
2102,76561198073642113,2014,207651,Utilities
2681,ScottyG555,2015,168314,Video Production
2381,Xyphien,2005,7296,Web Publishing
2382,Xyphien,2012,64657,Web Publishing


In [203]:
dff2.drop(index=dff2[dff2.Género=='N/D'].index,inplace=True)
dff2.drop(index=dff2[dff2.Año=='N/D'].index,inplace=True)

In [204]:
l1=[]
for a in dff2.loc[dff2.Género=='Action','Año']:
    if dff2.loc[(dff2.Año==a)&(dff2.Género=='Action'),'Horas_jugadas'].iloc[0]==0:
        continue
    else:
        horas=dff2.loc[(dff2.Año==a)&(dff2.Género=='Action'),'Horas_jugadas'].iloc[0]
        l1.append({'Año':int(a),'Horas':horas})
usuario=dff2.Id_usuario[dff2.Género=='Action'].unique()[0]
#"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}
print(f'"Usuario con más horas jugadas para Género Action: {usuario} Horas jugadas:{l1}"')

"Usuario con más horas jugadas para Género Action: Sp3ctre Horas jugadas:[{'Año': 1995, 'Horas': 217}, {'Año': 1999, 'Horas': 44}, {'Año': 2000, 'Horas': 70644}, {'Año': 2001, 'Horas': 13}, {'Año': 2002, 'Horas': 238}, {'Año': 2003, 'Horas': 7673}, {'Año': 2004, 'Horas': 127411}, {'Año': 2005, 'Horas': 21339}, {'Año': 2006, 'Horas': 896}, {'Año': 2007, 'Horas': 112784}, {'Año': 2008, 'Horas': 224}, {'Año': 2009, 'Horas': 108326}, {'Año': 2010, 'Horas': 78083}, {'Año': 2011, 'Horas': 154896}, {'Año': 2012, 'Horas': 378296}, {'Año': 2013, 'Horas': 202746}, {'Año': 2014, 'Horas': 130691}, {'Año': 2015, 'Horas': 313172}, {'Año': 2016, 'Horas': 29576}, {'Año': 2017, 'Horas': 43327}]"


In [205]:
dff2.to_csv('Función_UserForGenre.csv',index=False)

In [206]:
#dff2=pd.read_csv('Función_UserForGenre.csv')

In [207]:
#dff2.Género.unique().tolist()

In [208]:
def UserForGenre(genero):
    genero=genero.capitalize()
    l1=[]
    for a in dff2.loc[dff2.Género==genero,'Año']:
        if dff2.loc[(dff2.Año==a)&(dff2.Género==genero),'Horas_jugadas'].iloc[0]==0:
            continue
        else:
            horas=dff2.loc[(dff2.Año==a)&(dff2.Género==genero),'Horas_jugadas'].iloc[0]
            l1.append({'Año':int(a),'Horas':horas})
    usuario=dff2.Id_usuario[dff2.Género==genero].unique()[0]

    return {f"Usuario con más horas jugadas para Género {genero}:": usuario, 'Horas jugadas':l1}

    

In [209]:
print(UserForGenre('action'))

{'Usuario con más horas jugadas para Género Action:': 'Sp3ctre', 'Horas jugadas': [{'Año': 1995, 'Horas': 217}, {'Año': 1999, 'Horas': 44}, {'Año': 2000, 'Horas': 70644}, {'Año': 2001, 'Horas': 13}, {'Año': 2002, 'Horas': 238}, {'Año': 2003, 'Horas': 7673}, {'Año': 2004, 'Horas': 127411}, {'Año': 2005, 'Horas': 21339}, {'Año': 2006, 'Horas': 896}, {'Año': 2007, 'Horas': 112784}, {'Año': 2008, 'Horas': 224}, {'Año': 2009, 'Horas': 108326}, {'Año': 2010, 'Horas': 78083}, {'Año': 2011, 'Horas': 154896}, {'Año': 2012, 'Horas': 378296}, {'Año': 2013, 'Horas': 202746}, {'Año': 2014, 'Horas': 130691}, {'Año': 2015, 'Horas': 313172}, {'Año': 2016, 'Horas': 29576}, {'Año': 2017, 'Horas': 43327}]}


In [210]:
#def UsersRecommend( año : int ): #Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)
#Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [211]:
dfreviews.head()

,Id_juego,Recommend,Id_usuario,Año_review,sentiment_analysis
0,1250,True,76561197970982479,2011-01-01,2
1,22200,True,76561197970982479,2011-01-01,2
2,43110,True,76561197970982479,2011-01-01,1
3,251610,True,js41637,2014-01-01,2
4,227300,True,js41637,2013-01-01,1


In [212]:
dfreviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58431 entries, 0 to 58430
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Id_juego            58431 non-null  int64 
 1   Recommend           58431 non-null  bool  
 2   Id_usuario          58431 non-null  object
 3   Año_review          58431 non-null  object
 4   sentiment_analysis  58431 non-null  int64 
dtypes: bool(1), int64(2), object(2)
memory usage: 1.8+ MB


In [213]:
dfreviews['Año_review']=pd.to_datetime(dfreviews.Año_review)

In [214]:
dfreviewsf3=dfreviews.loc[(dfreviews.Recommend==True)&(dfreviews.sentiment_analysis==2)|(dfreviews.Recommend==True)&(dfreviews.sentiment_analysis==1),['Id_juego','Año_review']].copy()

In [215]:
años=dfreviews.Año_review.dt.year.sort_values().unique().tolist()
dff3=pd.DataFrame()
for a in años:
   dftemp1=dfreviewsf3.loc[(dfreviews.Año_review.dt.year==a)].value_counts(ascending=False).iloc[:3].to_frame().reset_index().copy()
   dff3=pd.concat([dff3,dftemp1])
dff3=dff3.reset_index(drop=True)
dff3

,Id_juego,Año_review,count
0,440,2010-01-01,10
1,1250,2010-01-01,6
2,630,2010-01-01,4
3,440,2011-01-01,79
4,620,2011-01-01,26
5,105600,2011-01-01,24
6,440,2012-01-01,260
7,105600,2012-01-01,41
8,4000,2012-01-01,38
9,440,2013-01-01,779


In [216]:
dff3=pd.merge(dfjuegos[['Id_juego','Nombre']],dff3).sort_values(by=['Año_review','count'],ascending=False).drop(columns='Id_juego')
dff3

,Nombre,Año_review,count
9,Counter-Strike: Global Offensive,2016-01-01,623
20,Team Fortress 2,2016-01-01,244
4,Garry's Mod,2016-01-01,140
8,Counter-Strike: Global Offensive,2015-01-01,1493
19,Team Fortress 2,2015-01-01,624
3,Garry's Mod,2015-01-01,347
18,Team Fortress 2,2014-01-01,1514
7,Counter-Strike: Global Offensive,2014-01-01,1047
2,Garry's Mod,2014-01-01,745
17,Team Fortress 2,2013-01-01,779


In [217]:
#convertimos la columna de años, a modo que solo quede el año numerico
dff3.Año_review=dff3.Año_review.dt.year

In [218]:
dff3.to_csv('Función_UsersRecommend.csv',index=False)

In [219]:
def UsersRecommend( año ):
    l=[]
    for a in range(3):
        dict={f'Puesto {a+1}':dff3[dff3.Año_review==año].iloc[a,0]}
        l.append(dict)
    return l
    
    #Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [220]:
print(UsersRecommend(2010))

[{'Puesto 1': 'Team Fortress 2'}, {'Puesto 2': 'Killing Floor'}, {'Puesto 3': 'Alien Swarm'}]


In [221]:
#def UsersWorstDeveloper( año : int ): #Devuelve el top 3 de desarrolladoras con juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)
#Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [222]:
dfreviewsf4=dfreviews.loc[(dfreviews.Recommend==False)&(dfreviews.sentiment_analysis==0),['Id_juego','Año_review']].copy()
dfreviewsf4.head()

,Id_juego,Año_review
62,359320,2015-01-01
132,344760,2015-01-01
398,400800,2016-01-01
526,215470,2013-01-01
537,226700,2014-01-01


In [223]:
dff4=dfreviewsf4.merge(dfjuegos[['Id_juego','Desarrollador']],how='left')
print(len(dff4))
dff4.head()

642


,Id_juego,Año_review,Desarrollador
0,359320,2015-01-01,N/D
1,344760,2015-01-01,Code}{atch
2,400800,2016-01-01,Milestone S.r.l.
3,215470,2013-01-01,Lukewarm Media
4,226700,2014-01-01,NaN


In [224]:
#haciendo webscrapping, con los datos que tenemos podemos sacar el genero faltante oara algunos juegos, ya que no todos nos arroja el resutlado de la pagina
for a in dff4.Id_juego[dff4.Desarrollador.isnull()].unique():
    url='https://store.steampowered.com/app/'+str(a)
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    elemento_div = soup.find('div', {'class': 'summary column', 'id': 'developers_list'})
    if elemento_div==None:
        continue
    else:
        texto_enlace = elemento_div.a.text
     
    dff4.loc[dff4.Id_juego==a,'Desarrollador']=texto_enlace

In [225]:
#ahora donde tenemos datos 'N/D' hacemos el mismo proceso
for a in dff4.Id_juego[dff4.Desarrollador=='N/D'].unique():
    url='https://store.steampowered.com/app/'+str(a)
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    elemento_div = soup.find('div', {'class': 'summary column', 'id': 'developers_list'})
    if elemento_div==None:
        continue
    else:
        texto_enlace = elemento_div.a.text
    dff4.loc[dff4.Id_juego==a,'Desarrollador']=texto_enlace

In [226]:
# despues comprobamos los valores que tenemos restantes
len(dff4[(dff4['Desarrollador'].isnull())|(dff4['Desarrollador']=='N/D')])


57

In [227]:
dff4

,Id_juego,Año_review,Desarrollador
0,359320,2015-01-01,Frontier Developments
1,344760,2015-01-01,Code}{atch
2,400800,2016-01-01,Milestone S.r.l.
3,215470,2013-01-01,Lukewarm Media
4,226700,2014-01-01,Fredaikis AB
...,...,...,...
637,304930,2015-01-01,Smartly Dressed Games
638,209630,2014-01-01,Frogwares
639,370240,2015-01-01,Visual Concepts
640,391460,2016-01-01,WARTEAM


In [228]:
# quitamos la columna Id_juego que ya no utlizaremos
dff4.drop(columns='Id_juego',inplace=True)

In [229]:
# se realiza un Dataframe con los datos necesarios para la función
dff4_final=pd.DataFrame()
for a in dff4.Año_review.unique():
    dff4_final=pd.concat([dff4_final,dff4[dff4.Año_review==a].value_counts(ascending=False).reset_index()[:3]])


In [230]:
dff4_final

,Año_review,Desarrollador,count
0,2015-01-01,Valve,14
1,2015-01-01,OVERKILL - a Starbreeze Studio.,9
2,2015-01-01,Facepunch Studios,7
0,2016-01-01,Valve,17
1,2016-01-01,Hello Games,10
2,2016-01-01,Bohemia Interactive,6
0,2013-01-01,Infinity Ward,3
1,2013-01-01,Valve,2
2,2013-01-01,Lukewarm Media,1
0,2014-01-01,Valve,11


In [231]:
dff4_final

,Año_review,Desarrollador,count
0,2015-01-01,Valve,14
1,2015-01-01,OVERKILL - a Starbreeze Studio.,9
2,2015-01-01,Facepunch Studios,7
0,2016-01-01,Valve,17
1,2016-01-01,Hello Games,10
2,2016-01-01,Bohemia Interactive,6
0,2013-01-01,Infinity Ward,3
1,2013-01-01,Valve,2
2,2013-01-01,Lukewarm Media,1
0,2014-01-01,Valve,11


In [232]:
dff4_final['Año_review']=dff4_final.Año_review.dt.year

In [233]:
#guardamos en una funcion los datos para nuestra funcion
dff4_final.to_csv('Función_UsersWorstDeveloper.csv',index=False)

In [234]:
def UsersWorstDeveloper( año ):
    dict={}
    for a,b in enumerate(dff4_final.loc[dff4_final.Año_review==año,'Desarrollador']):
        dict['Puesto '+str(a+1)]=b
    return [dict]


In [235]:
UsersWorstDeveloper(2012)

[{'Puesto 1': 'Valve'}]

In [236]:
#def sentiment_analysis( empresa desarrolladora : str ): #Según la empresa desarrolladora, se devuelve un diccionario con el nombre de la desarrolladora como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor.
#Ejemplo de retorno: {'Valve' : [Negative = 182, Neutral = 120, Positive = 278]}

In [237]:
#seleccionamos las columnas de reviews que usaremos
dfreviewsf5=dfreviews[['Id_juego','sentiment_analysis']].copy()
dfreviewsf5.head()

,Id_juego,sentiment_analysis
0,1250,2
1,22200,2
2,43110,1
3,251610,2
4,227300,1


In [238]:
#agregamos las columnas de Desarrollador del dataframe de juegos
dfreviewsf5=pd.merge(dfjuegos[['Id_juego','Desarrollador']],dfreviewsf5,how='right')
dfreviewsf5.head()

,Id_juego,Desarrollador,sentiment_analysis
0,1250,Tripwire Interactive,2
1,22200,ACE Team,2
2,43110,NaN,1
3,251610,NaN,2
4,227300,SCS Software,1


In [239]:
#hacemos un dataframe 
dff5=pd.get_dummies(dfreviewsf5,columns=['sentiment_analysis'],dtype=int).drop(columns='Id_juego').groupby('Desarrollador').count().reset_index()

In [240]:
dff5['Desarrollador']=dff5['Desarrollador'].str.capitalize()

In [241]:
# la columna Desarrollador tiene valores que no deberian estar, como (Mac), lo cual afecta a los resultados de la agurapcion se 
l=[]
for a in dff5.Desarrollador:
    b=0
    c=0    
    if '(' in a:
        b=a.find('(')
    if ')' in a:
        c=a.find(')')
        if 'mac' in a[b:c+1] and  a[b:c+1] not in l:
            l.append(a[b:c+1])
        if 'linux' in a[b:c+1] and a[b:c+1] not in l:
            l.append(a[b:c+1])
l

['(mac &amp; linux)',
 '(mac and linux)',
 '(mac,linux)',
 '(mac)',
 '(mac/linux)',
 '(linux)',
 '(mac, linux)',
 '(mac, linux, &amp; windows update)']

In [242]:
for a in l:
    dff5.loc[dff5.Desarrollador.str.contains(a,regex=False),'Desarrollador']=dff5.loc[dff5.Desarrollador.str.contains(a,regex=False),'Desarrollador'].str.replace(a,'')

In [243]:
l=[]
for a in dff5.Desarrollador:
    b=0
    c=0    
    if '(' in a:
        b=a.find('(')
    if ')' in a:
        c=a.find(')')
        if 'mac' in a[b:c+1] and  a[b:c+1] not in l:
            l.append(a[b:c+1])
        if 'linux' in a[b:c+1] and a[b:c+1] not in l:
            l.append(a[b:c+1])
l

[]

In [244]:
dff5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2121 entries, 0 to 2120
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Desarrollador         2121 non-null   object
 1   sentiment_analysis_0  2121 non-null   int64 
 2   sentiment_analysis_1  2121 non-null   int64 
 3   sentiment_analysis_2  2121 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 66.4+ KB


In [245]:
#cambiamos manualmente los campos donde aparece ubisoft, por ser un desarrollador importante
dff5.loc[dff5.Desarrollador.str.contains('Ubisoft'),'Desarrollador']='Ubisoft'
dff5.loc[dff5.Desarrollador.str.contains(' in collaboration with ubisoft reflections'),'Desarrollador']=dff5.loc[dff5.Desarrollador.str.contains(' in collaboration with ubisoft reflections'),'Desarrollador'].str.replace(' in collaboration with ubisoft reflections','ubisoft')
dff5.loc[dff5.Desarrollador.str.contains('ubisoft montreal'),'Desarrollador']=dff5.loc[dff5.Desarrollador.str.contains('ubisoft montreal'),'Desarrollador'].str.replace('ubisoft montreal','ubisoft')
dff5.loc[dff5.Desarrollador.str.contains('ubisoft  shanghaï'),'Desarrollador']=dff5.loc[dff5.Desarrollador.str.contains('ubisoft  shanghaï'),'Desarrollador'].str.replace('ubisoft  shanghaï','ubisoft')
dff5.loc[dff5.Desarrollador.str.contains('ubisoft kiev'),'Desarrollador']=dff5.loc[dff5.Desarrollador.str.contains('ubisoft kiev'),'Desarrollador'].str.replace('ubisoft kiev','ubisoft')

In [246]:
dff5_prueba=dff5.copy()

In [247]:
dff5_prueba.loc[:,'Desarrollador']=dff5.Desarrollador.str.split(',')

In [248]:
dff5_prueba=dff5_prueba.explode('Desarrollador')

In [249]:
dff5_prueba.Desarrollador=dff5_prueba.Desarrollador.str.strip('. ').str.capitalize()
dff5_prueba.Desarrollador=dff5_prueba.Desarrollador.str.strip('. ').str.capitalize()

In [250]:
dff5_prueba.loc[dff5_prueba.Desarrollador.str.contains('quare'),'Desarrollador']='Square enix'
l=dff5_prueba.Desarrollador[dff5_prueba.Desarrollador.str.len()<4].unique().tolist()

In [251]:
l.remove('Rex')
l.remove('橘子班')
l.remove('インレ')
l.remove('773')
l.remove('D3t')
l.remove('Gun')
l.remove('Kog')
l.remove('M2h')


In [252]:
for a in l:
    dff5_prueba.drop(index=dff5_prueba.loc[dff5_prueba.Desarrollador==a].index,inplace=True)

In [253]:
dff5_prueba=dff5_prueba.groupby(by='Desarrollador').sum().reset_index()

In [254]:
dff5_prueba.Desarrollador[dff5_prueba.Desarrollador.str.len()<4].unique()

array(['773', 'D3t', 'Gun', 'Kog', 'M2h', 'Rex', 'インレ', '橘子班'],
      dtype=object)

In [255]:
for a in dff5_prueba.Desarrollador:
    print(a)

07th expansion
10th art studio
10tons ltd
11 bit studios
14° east
16bit nights
17-bit
1c company
1c-avalon
1c-softclub
1c: maddox games
1c:inoco
22cans
24 caret games
2d boy
2darray
2dengine.com
2k australia
2k boston
2k china
2k czech
2k marin
2play
2x2 games
3 sprockets
30.06 studios ltd
3000ad
343 industries
38 studios
3909
3d realms (apogee software)
3f interactive
4 i lab
48h studio
49 games
4a games
5 bits games
5 lives studios
5th cell media
5wolf
773
777 studios
8-bit studio
800 north and digital ranch
8888888 labs
88mm
8monkey labs
9heads game studios
@unepic_fran
A barnett
A crowd of monsters
A jolly corpse
Abbey games
Above and beyond technologies
Abraham carreola
Abrakam sa
Abstract tree studio
Abstraction games
Access games
Ace maddox
Ace team
Acid nerve
Acquire corp
Actalogic
Adam foster
Adrian zingg
Adventure planning service
Adventure productions
Aeon dream studios
Afterthought llc
Airtight games
Aivaras klimas
Akakiy petrushkin
Alan edwardes
Alan zucconi
Alawar
Alawar 

In [256]:
#guardamos en un csv los datos para la función
dff5_prueba.to_csv('Función_sentiment_analysis.csv', index=False)

In [257]:
def sentiment_analysis( empresa_desarrolladora ): 
#Según la empresa desarrolladora, se devuelve un diccionario con el nombre de la desarrolladora como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor.
#Ejemplo de retorno: {'Valve' : [Negative = 182, Neutral = 120, Positive = 278]}
   Negative=dff5.sentiment_analysis_0[dff5.Desarrollador==empresa_desarrolladora].iloc[0]
   Neutral=dff5.sentiment_analysis_1[dff5.Desarrollador==empresa_desarrolladora].iloc[0]
   Positive = dff5.sentiment_analysis_2[dff5.Desarrollador==empresa_desarrolladora].iloc[0]
   l= f'[Negative = {Negative}, Neutral =  {Neutral}, Positive = {Positive}]'
   return {empresa_desarrolladora:l}

In [ ]:
sentiment_analysis('Valve')

{'Valve': '[Negative = 9586, Neutral =  9586, Positive = 9586]'}